In [11]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import matplotlib.pyplot as plt
from transforms3d.euler import euler2mat
from mpl_toolkits.mplot3d import Axes3D
import pickle
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd
import requests
import csv
import torch

from Joint import *
from Viewer import *
from amc_parser import *

def generate_np_data(amc_path:str, asf_path:str):
    joints = parse_asf(asf_path)
    motions = parse_amc(amc_path)

    one_motion_list = []
    for i in range(len(motions)):
        joints['root'].set_motion(motions[i])

        single_row = []
        for j in joints.values():
            single_row.append(j.coordinate)
        single_row_np = np.concatenate(single_row)
        one_motion_list.append(single_row_np)
    return np.stack(one_motion_list)

final_df_output_name = 'data/df_amc_full.pkl'
src_dir = './data/subjects/'

with open('./data/dataset_id_desc.pkl', 'rb') as f:
    id_desc = pickle.load(f)
with open('./data/dataset_id_trial_desc.pkl', 'rb') as f:
    id_trial_desc = pickle.load(f)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
df_list = []; column_names = ['main_id', 'trial_id', 'id_desc', 'id_trial_desc', 'data']
for sub in tqdm(sorted(os.listdir(src_dir))):
    if '.' in sub: continue
    main_id = int(sub)
    files = os.listdir(src_dir + sub)
    asf_path = os.path.join(*[src_dir, sub, [f for f in files if f.endswith('.asf')][0] ])
    #print(asf_path)
    files = [f for f in files if f.endswith('.amc')]
    for f in sorted(files):
        trial_id = int(f.rsplit('.')[0].split('_')[1])
        amc_path = os.path.join(*[src_dir, sub, f])
        np_data = generate_np_data(amc_path=amc_path, asf_path=asf_path)
        if main_id in id_desc.index:
            id_desc_str = id_desc.at[main_id, 'motion_description']
        else:
            id_desc_str = ''
        if (main_id, trial_id) in id_trial_desc.index:
            id_trial_desc_str = id_trial_desc.at[(main_id, trial_id), 'motion_description']
        else:
            id_trial_desc_str = ''
        df_list.append(
            [main_id, trial_id, id_desc_str, id_trial_desc_str, np_data]
        )
        #print(amc_path, main_id, trial_id, id_desc_str, id_trial_desc_str)

100%|█████████████████████████████████████████████████████████████████████████████████| 113/113 [1:25:47<00:00, 45.55s/it]


In [14]:
df = pd.DataFrame(df_list, columns=column_names)
df.set_index(['main_id', 'trial_id'], inplace=True)
df.to_pickle(final_df_output_name)
df.head(4)

id_desc  \
main_id trial_id                                                      
1       1         climb, swing, hang on playground equipment (14...   
        2         climb, swing, hang on playground equipment (14...   
        3         climb, swing, hang on playground equipment (14...   
        4         climb, swing, hang on playground equipment (14...   
        5         climb, swing, hang on playground equipment (14...   
...                                                             ...   
94      12                                indian dance (16 trials)    
        13                                indian dance (16 trials)    
        14                                indian dance (16 trials)    
        15                                indian dance (16 trials)    
        16                                indian dance (16 trials)    

                                            id_trial_desc  \
main_id trial_id                                            
1       1         playground - forward jumps, turn around   
        2                              playground - climb   
        3                 playground - climb, hang, swing   
        4                              playground - climb   
        5                    playground - climb, go under   
...                                                   ...   
94      12                                             \n   
        13                                             \n   
        14                                             \n   
        15                                             \n   
        16                                             \n   

                                                               data  
main_id trial_id                                                     
1       1         [[[9.37216], [17.8693], [-17.3198], [10.502614...  
        2         [[[-7.4145], [17.6804], [24.0195], [-8.7902143...  
        3         [[[-0.808516], [17.8049], [18.3516], [-1.84392...  
        4         [[[12.7274], [17.7984], [16.0108], [11.9998742...  
        5         [[[15.19], [17.8785], [17.1506], [14.269579497...  
...                                                             ...  
94      12        [[[-24.743], [15.4761], [18.0211], [-25.863505...  
        13        [[[11.2094], [15.5233], [1.18117], [10.5713228...  
        14        [[[8.10005], [12.2714], [1.87369], [7.31143587...  
        15        [[[44.8411], [12.7071], [14.1774], [45.5171723...  
        16        [[[7.47009], [15.673], [0.286588], [6.84211830...  

[2514 rows x 3 columns]